In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

import re
import string
import pandas as pd
import spacy
import tqdm

from tqdm._tqdm_notebook import tqdm_notebook

tqdm_notebook.pandas()

C:\ProgramData\Miniconda3\envs\digM\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  import sys
C:\ProgramData\Miniconda3\envs\digM\lib\site-packages\tqdm\std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
nlp = spacy.load('en_core_web_sm')
tokenizer = nlp.Defaults.create_tokenizer(nlp)
stop_words = nlp.Defaults.stop_words

In [3]:
df = pd.read_csv('../data/Dataset.csv')
df.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [4]:
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [5]:
def clean_text(text: str):
    text = re.sub(r'<[^>]*>', ' ', text)  # remove <br /> tags
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    return text

def preprocess_text(text: str):
    sentence = list()
    doc = nlp(text)
    for word in doc:
        sentence.append(word.lemma_)
    return ' '.join(sentence)

In [6]:
df['cleaned_review'] = df['review'].progress_apply(lambda x:clean_text(x))

In [7]:
df['cleaned_review'] = df['cleaned_review'].progress_apply(lambda x:preprocess_text(x))

df.tail()

,review,sentiment,cleaned_review
49995,I thought this movie did a down right good job...,positive,-PRON- think this movie do a down right good j...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogue bad act idiotic direct t...
49997,I am a Catholic taught in parochial elementary...,negative,-PRON- be a Catholic teach in parochial elemen...
49998,I'm going to have to disagree with the previou...,negative,-PRON- be go to have to disagree with the prev...
49999,No one expects the Star Trek movies to be high...,negative,no one expect the Star Trek movie to be high a...


In [8]:
df['cleaned_review'] = df['cleaned_review'].str.replace('-PRON-', '')
df.tail()

,review,sentiment,cleaned_review
49995,I thought this movie did a down right good job...,positive,think this movie do a down right good job be...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogue bad act idiotic direct t...
49997,I am a Catholic taught in parochial elementary...,negative,be a Catholic teach in parochial elementary s...
49998,I'm going to have to disagree with the previou...,negative,be go to have to disagree with the previous c...
49999,No one expects the Star Trek movies to be high...,negative,no one expect the Star Trek movie to be high a...


In [11]:
X = df['cleaned_review']
y = df['sentiment']

final_df = pd.concat([X, y], axis=1)
final_df.tail()

,cleaned_review,sentiment
49995,think this movie do a down right good job be...,positive
49996,bad plot bad dialogue bad act idiotic direct t...,negative
49997,be a Catholic teach in parochial elementary s...,negative
49998,be go to have to disagree with the previous c...,negative
49999,no one expect the Star Trek movie to be high a...,negative


In [13]:
final_df.to_csv('../data/final_data.csv', index=False)